## Problema

Predecir la probabilidad de que un comentario sea tóxico, comentartio tóxico "1", comentario no tóxico "0"
Test: Todos los comentarios clasificados como "1" ó "0"


In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [2]:
# Load datasets

import pandas as pd
import os



val_data = pd.read_csv("validation.csv")
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("jigsaw-toxic-comment-train.csv")

In [3]:
# Preview train set
train_data.sample(100)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
198221,9a0f3ddd5a242ffb,"Stop fooling yourself Narek, and stop being ag...",0,0,0,0,0,0
63079,a8c0c22da9328590,Will you lot frikking stop it \n\nUse any fook...,1,0,0,0,0,0
35274,5e2ec333c94e9a40,"""\n\nHeh. I created the article on Jhereg, fol...",0,0,0,0,0,0
195666,904764c2ea6efe97,*Support seems to be the most sensible option.,0,0,0,0,0,0
46663,7cb7f0d6df9ca841,"""\n Speedy deletion of """"John falls"""" \n\n A p...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
155545,bfb4a1ab5d7e3028,sajad \n\nselena i have good sound as you.but ...,0,0,0,0,0,0
204509,b3025e5fabed2ea8,"Hey, Beckerhead, go f yourself.",0,0,0,0,0,0
196663,941e60ce2ccf115e,":Oh no!! Thank you for informing me, I'll do a...",0,0,0,0,0,0
197938,98f1fb1b1161084f,"== Essay == \n\n Hi Jehochman, I've written an...",0,0,0,0,0,0


In [4]:
val_data.sample(5)

,id,comment_text,lang,toxic
4115,4115,"Merhaba, Essex County bu maddeyi Essex County(...",tr,0
2909,2909,...por esto y por esto. Ya he sacado Calígula ...,es,0
1185,1185,"da niente a troppo che esagerato, adesso sei a...",it,1
6060,6060,Coincido con lo dicho por Alelapenya. Espero q...,es,0
5065,5065,No insistas vos en cagarte en mi trabajo. Ya ...,es,0


In [5]:
test_data.sample(5)

,id,content,lang
20899,20899,"ah, wow. stai attento che se in qualche partit...",it
57826,57826,"Arthemius x, o artigo Cronica está sendo vanda...",pt
63467,63467,Mil perdões!!! Erro absurdo meu com certeza. F...,pt
3998,3998,"Te meterán en la hoguera por hereje,:) ya mism...",es
28350,28350,No sabía que pudieras ver el futuro. Me dices ...,es


In [6]:
# Remover 
import re

val = val_data
train = train_data

def clean(text):
    # fill the missing entries and convert them to lower case
    text = text.fillna("fillna").str.lower()
    # replace the newline characters with space 
    text = text.map(lambda x: re.sub('\\n',' ',str(x)))
    text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
    # remove usernames and links
    text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    return text

val["comment_text"] = clean(val["comment_text"])
test_data["content"] = clean(test_data["content"])
train["comment_text"] = clean(train["comment_text"])


In [ ]:
pip install transformers

In [11]:
# Load DistilBERT tokenizer
import transformers

tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [12]:
# Segregate the comments and their labels (not applicable for test set)
train_comments = train.comment_text.astype(str).values
val_comments = val_data.comment_text.astype(str).values
test_comments = test_data.content.astype(str).values

y_valid = val.toxic.values
y_train = train.toxic.values

In [13]:
import gc
gc.collect()

40

In [9]:
#pip install -q googletrans

In [14]:
# Demo examples of translations
from googletrans import Translator

sample_comment = test_comments[48649]
print("Original comment:", sample_comment)
translated_comment = Translator().translate(sample_comment)
print("\n")
print("Translated comment:", translated_comment.text)

Original comment:  ¡ah! sí, ya lo sé... pero como que no puedo sacarme ciertos argentinismos de encima a la hora de escribir. —   kved    (discusión)    pd: aunque no sé si lo correcto no es escribir  bloqueé  en lugar de  bloquee . para solucionar ese tema, es más fácil decir   bloquié   y que la rae se vaya a tomar por culo.   ;)


Translated comment: Ah! Yes, I know ... but I can not get me out certain argentinismos off when writing. - kved (discussion) pd: I do not know if right not write blocked instead of blocking. to solve this issue, it is easier to say rae bloquié and that is to take the ass. ;)


In [15]:
test_comments

array(['doctor who adlı viki başlığına 12. doctor olarak bir viki yazarı kendi adını eklemiştir. şahsen düzelttim. onaylarsanız sevinirim. occipital ',
       ' вполне возможно, но я пока не вижу необходимости выделять материал в отдельную статью. если про правосудие в ссср будет написано хотя бы килобайт 20-30 — тогда да, следует разделить. пока же мы в итоге получим одну куцую статью правосудие и другую не менее куцую статью правосудие в ссср. мне кажется, что этот вопрос вполне разумно решать на основе правил вп:размер статей? которые не предписывают разделения, пока размер статьи не достигнет хотя бы 50 тыс. знаков. ',
       'quindi tu sei uno di quelli   conservativi  , che preferiscono non cancellare. ok. avresti lasciato anche   sfaccimma  ? si? ok. contento te... io non approvo per nulla, ma non conto nemmeno nulla... allora lo sai che faccio? me ne frego! (aborro il fascismo, ma quando ce vo , ce vo !) elborgo (sms) ',
       ...,
       'innnazitutto ti ringrazio per i ringr